# The Eight Schools Problem: A Hierarchical Bayesian Analysis

The "eight schools problem" is a very famous example problem that is often used to illustrate hierarchical models and the Bayesian inference process.  This problem was originally described in a 1981 paper by TODO ref, and it has prominently been features as an example in the third edition of Bayesian Data Analysis (TODO reference), to which I will refer below as "BDA3."  The "full Bayesian" model that is commonly used to analyze this problem is simple but very generalizable, and a good understanding of how this model is constructed and analyzed provides a lot of insight into Bayesian data analysis.  Unfortunately, while there are several STAN implementations of this problem available on the internet, none of them provide a lot of context about the problem and simply refer the reader to BDA3.  The **purpose of this document** is to give a reasonably in-depth description of the eight schools problem and the hierarchical model that is commonly used to analyze it.  The document includes:

- An introduction of the eight schools problem and associated data


- A defintion of the hierarchical model used to do analyses


- A partial analytical derivation of the full model posterior


- A PySTAN implementation of the model



## The Effects of SAT-V Coaching in Eight Schools 

Eight schools hosted special coaching programs for the verbal section of the Scholastic Aptitude Test (SAT-V), and a study was done to investigate the effects of these programs.  For each school, a linear regression was performed where student SAT-V scores were regressed on a binary variable indicating treatment group, with student scores on multiple PSAT sections included as additional controls.  The effects measured by these regresions, along with their sample variances - which are assumed known - are summarized in this table:


| School | Estimated Treatment Effect | Effect Standard Error |
| --- | --- | --- |
| 1 | 28 | 15 |
| 2 | 8 | 10 |
| 3 | -3 | 16 |
| 4 | 7 | 11 |
| 5 | -1 | 9 |
| 6 | 1 | 11 |
| 7 | 18 | 10 |
| 8 | 12 | 18 |


The problem we now face is how to interpret these results and use them to do inference.  One thing we could do is to assume that there is no difference in the effect between schools, and therefore that each school's measured effect is an independent estimate of the one "common" effect $\theta$.  Suppose that we do this and perform a simple Bayesian analysis where our estimates across schools are assumed to be normally distributed and a noninformative (uniform) prior distribution is given to $\theta$.  We will get that our posterior mean for $\theta$ is ~7.7.  Consider a second simple strategy where we assume that each school's effect is different, and we only use data from a given school to estimate the effect at that school.  A simple Bayesian analysis for each school yields school level posterior means that are close to the observed effect sizes, but which are essentially indistinguishable based on the widths of the posterior credible intervals.

These two strategies are nice because of their simplicity, but they are each rather blunt.  The first uses a complete pooling of information across schools, and the second does not allow for any information sharing across schools.  These two strategies also give very different estimates for some of the school level effects.  For example, the first simple strategy estimates the posterior mean of the effect at school 1 as $\approx 7.7$ (the pooled estimate given to all schools), while the second simple strategy estimates it as $\approx 28$.  This result motivates the use of a slightly more complicated model that allows for the "partial" sharing of information across schools...


## Hierarchical Generative Model

Let $\bar{y}_{.j}$ denote the estimated treatment effect for school $j$, where $j=1, ..., 8$.  Let $\sigma_j^2$ denote the sampling variance associated with the estimate at school $j$.  Note that this notation is used to maintain consistency with the presentation of this problem in BDA3, and the "dot" notation used to denote the within group sample mean is classically used in analysis of variance.  We now assume the following hierarchical model describing the generative process that creates these $\theta_j$ values:

$$
\overline{y_{.j}} \sim N(\theta_j, \sigma_j^2)
$$

Where:

$$
\overline{y_{.j}} = \frac{1}{n_j} \sum_{i=1}^{n_j} y_{ij}
$$
$$
\sigma_j^2 = \frac{\sigma^2}{n_j}
$$

Here comes the hierarchical part - we assume that the $\theta_j$ values are themselves sampled independently from a Normal distribution with shared hyperparameters $\mu$ and $\tau$:

$$
\theta_j \sim_{iid} N(\mu, \tau^2)
$$


## Posterior Distribution over Model Parameters

Using a Bayesian approach with the model specified above, the full posterior distribution over all parameters can be factorized using the probability chain rule:

$p(\theta, \mu, \tau | y) = p(\theta|\mu, \tau, y)p(\mu|\tau, y)p(\tau|y)$


Specifying $p(\mu, \tau) = p(\mu | \tau)p(\tau)$ - the prior distribution over our hyperparameters - will give us enough information to analytically solve for the posterior.  We will do this below as we derive the second and third terms of the factorization.  We don't need any information from the prior in order to derive the first term $p(\theta|\mu, \tau, y)$, since we are already conditioning on both $\mu$ and $\tau$.  Our model assumptions give us a prior for each $\theta_j$, and a sampling distribution of $y_{ij}$ given $\theta_j$ (a likelihood).  Using these to solve for the posterior on $\theta$ yields:

$$
\theta|\mu, \tau, y \sim N(\hat{\theta}_j, V_j)
$$

Where:

$$
\hat{\theta}_j = \frac{\frac{1}{\sigma_j^2} \bar{y}_{.j} + \frac{1}{\tau^2} \mu  }{\frac{1}{\sigma_j^2} + \frac{1}{\tau^2}}
$$

$$
V_j = \frac{1}{ \frac{1}{\sigma_j^2} + \frac{1}{\tau^2}  }
$$

Now, we **assume a uniform conditional prior** on $\mu|\tau$, or in other words assume that $p(\mu | \tau) = c$.  This allows us to factor the joint prior over these hyperparameters as $p(\mu, \tau) = p(\mu|\tau)p(\tau) \propto p(\tau)$.  The assignment of a "noninformative" prior over $\mu|\tau$ is a **modeling decision** that is motivated by the fact that the data we observe will provide us with a lot of information about $\mu$ (pg. 115).  This decision implies the following conditional distribution for $\mu|\tau, y$, the second term in the factorization of the posterior:

$$
\mu|\tau, y \sim N(\hat{\mu}, V_\mu)
$$

Where:

$$
\hat{\mu} = \frac{  \sum_{j=1}^J \frac{1}{\sigma^2_j + \tau^2} \bar{y}_{.j}    }{  \sum_{j=1}^J \frac{1}{\sigma^2_j + \tau^2}  }
$$

$$
V_\mu^{-1} = \sum_{j=1}^J \frac{1}{\sigma_j^2 + \tau^2}
$$


Finally, we deterine the third term of the factorization, the conditional posterior of $\tau|y$.  We can express this as:

$$
p(\tau|y) = \frac{p(\mu, \tau|y)}{p(\mu|\tau, y)}
$$



$$
\propto \frac{p(\tau) \prod_{j=1}^J N(\bar{y}_{.j}|\hat{\mu}, \sigma_j^2 + \tau^2) }{N(\hat{\mu}|\hat{\mu}, V_u)}
$$
$$
\propto p(\tau) V_u^{1/2} \prod_{j=1}^J (\sigma_j^2 + \tau^2)^{-1/2} exp\left(  - \frac{(\bar{y}_{.j} - \hat{\mu})^2} {2(\sigma_j^2 + \tau^2)} \right)
$$

We **assume a uniform prior** on $\tau$: $p(\tau) \propto c$ for $\tau > 0$.  This implies:

$$
p(\tau|y) \propto V_u^{1/2} \prod_{j=1}^J (\sigma_j^2 + \tau^2)^{-1/2} exp\left(  - \frac{(\bar{y}_{.j} - \hat{\mu})^2} {2(\sigma_j^2 + \tau^2)} \right)
$$

We now have an analytical expression (up to constant scaling) of our posterior $p(\theta, \mu, \tau | y)$.  We see that the posterior is quite a complicated function, and instead of attempting to integrate it (e.g. to normalize it) we will prefer to analyze it numerically.  This can be done in a straightforward way - we first sample $\tau$, which can be done by computing $p(\tau|y)$ for a uniformly spaced grid of $\tau$ values and using the inverse CDF method (see [here](https://stephens999.github.io/fiveMinuteStats/inverse_transform_sampling.html)) on this discretized distribution of $\tau$.  We then sample $\mu$ and then $\theta$ from their associated Normal distributions.

--------------------------------------------

To summarize, we:

- expressed the posterior distribution for the parameters of our hierarchical model as a factorization of three conditional posteriors

- Assumed noninformative uniform priors for $p(\mu|\tau)$ and $p(\tau)$

- Used the the assumed priors to analytically express the posterior factors


The full details of the posterior derivation are omitted here for the sake of brevity, but are included in BDA3. (TODO REF) 

## Tau and Pooled vs. Separate School Estimates  

The procedure described above allows us to do inference on the $\tau$ parameter of the model after specifying a prior distribution over it.  However, if we were to set $\tau = 0$ we would find that the distributions for the $\theta_j$ values given by our Bayesian procedure would all be centered around the pooled estimate of $\theta_j = 7.7$ that is given by the the first "simple strategy" described above.  On the other hand, if we take $\tau \rightarrow \infty$, we find that the distribution estimates over $\theta_j$ parameters are centered about the separate estimates given by the second "simple strategy."  We can thus think of $\tau$ as a sort of knob that we could turn to give us a different interpolation between the fully pooled and fully separated strategies for estimating the $\theta_j$ values.  This result makes sense given the interpretation of $\tau$ in the model - small values of $\tau$ imply that the different school effects are (in all probability) very close to each other.


This is mentioned only as a curiosity, but it illustrates the flexibility and power of the hierarchical model we've constructed to describe this scenario.




## Automatically Sampling from the Posterior with STAN

In BDA3, Gelman et. all mention the sampling method described above as a way to numerically analyze the posterior distribution of the model (pg. 118).  However, in general we can avoid the posterior derivations necessary for this by using a dedicated Bayesian software package like STAN.  STAN allows us to build a model by declaring a likelihood and prior, and then automatically do inference on the posterior after supplying data to the model.  STAN is very powerful in that it allows us to easily experiment with different model specifications without having to spend time coming up with a good way to sample from the posterior - it all happens "automagically."  The magic is supplied by way of a powerful Markov Chain Monte Carlo algorithm known as Hamiltonian Monte Carlo, which is how STAN draws samples.  STAN has APIs for R and Python (among other languages), and below is included an example of using stan through the PyStan Python package.  Here are some helpful resources related to STAN and PyStan:


- The Stan User Manual is [here](https://mc-stan.org/docs/2_29/stan-users-guide-2_29.pdf).


- A wonderful paper that gives some intuition for Hamiltonian Monte Carlo sampling is [here](https://arxiv.org/pdf/1701.02434.pdf?ref=https://githubhelp.com)


- The PyStan GitHub repository (which includes some documentation) can be [here](https://github.com/stan-dev/pystan).



Below is a PyStan implementation of the eight schools problem described in detail above.  TODO finish blurb

In [3]:
import pystan
import pandas as pd


model_code = """
data {
  int<lower=0> J;
  real y[J];
  real<lower=0> sigma[J];
}

parameters {
  real mu;
  real<lower=0> tau;
  real theta[J];
}

model {
  mu ~ uniform(-20, 20);
  tau ~ uniform(0, 50);
  theta ~ normal(mu, tau);
  y ~ normal(theta, sigma);
}
"""

model_data = dict(J = 8, y = [28,  8, -3,  7, -1,  1, 18, 12], 
sigma = [15, 10, 16, 11,  9, 11, 10, 18])



In [1]:
import pystan
import pandas as pd


model_code = """
data {
  int<lower=0> J;
  real y[J];
  real<lower=0> sigma[J];
  real<lower=0> tau;
}

parameters {
  real mu;
  real theta[J];
}

model {
  theta ~ normal(mu, tau);
  y ~ normal(theta, sigma);
  mu ~ uniform(-20, 20);
}
"""

model_data = dict(J = 8, y = [28,  8, -3,  7, -1,  1, 18, 12], 
sigma = [15, 10, 16, 11,  9, 11, 10, 18])



In [2]:
model_data["tau"] = 8

model_data

{'J': 8,
 'y': [28, 8, -3, 7, -1, 1, 18, 12],
 'sigma': [15, 10, 16, 11, 9, 11, 10, 18],
 'tau': 8}

In [3]:
# fit = pystan.stan(
#     model_code=model_code, data=model_data, iter=1000, chains=1
# )

# compile model
sm = pystan.StanModel(model_code=model_code)

# Train the model and generate samples
fit = sm.sampling(
    data=model_data,
    iter=1000,
    chains=4,
    warmup=500,
    thin=1,
    control=dict(adapt_delta=0.8),
    seed=101
)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_04b6ca40ee67b46c0e85084ce20ced4f NOW.


In [29]:
summary_dict = fit.summary()
df = pd.DataFrame(summary_dict['summary'], 
                  columns=summary_dict['summary_colnames'], 
                  index=summary_dict['summary_rownames'])


df

,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
mu,8.031578,0.333714,4.061936,-0.703092,5.589061,8.048183,10.684351,15.842837,148.154933,1.036538
theta[1],8.138984,0.335126,4.160210,-0.834255,5.490742,8.147732,10.975592,16.493141,154.104432,1.033676
theta[2],8.037414,0.338071,4.149238,-0.756605,5.413917,8.015429,10.695799,16.216609,150.633343,1.035873
theta[3],7.968147,0.329338,4.156863,-0.803777,5.364213,7.969778,10.762491,15.867992,159.311289,1.033385
theta[4],7.990305,0.337747,4.150151,-0.546746,5.401063,8.060295,10.682770,15.966547,150.989121,1.033497
theta[5],7.905201,0.335611,4.137315,-1.045362,5.277481,7.941587,10.721696,15.857161,151.972376,1.035300
theta[6],7.957442,0.330549,4.161165,-0.896794,5.381397,7.960650,10.699240,16.176975,158.473788,1.033722
theta[7],8.112797,0.330533,4.132082,-0.598849,5.562385,8.042120,10.790248,15.955197,156.281543,1.032959
theta[8],8.049164,0.341033,4.167562,-0.471013,5.497870,8.099621,10.730022,15.954249,149.338758,1.035368
lp__,-6.711166,0.091008,2.068612,-11.632739,-7.883887,-6.422469,-5.168679,-3.593750,516.651844,1.008145


## References

TODO!